In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import resnet18, ResNet50_Weights
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import pandas as pd
import os
from pts_loader import load



In [2]:
image_dir = "Images/"
train_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/train_1.txt"
test_txt = "train_test_files/5_folders_cross_validations_files/cross_validation_1/test_1.txt"


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
])

In [6]:
def load_data(txt_file, image_dir):
    data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
    image_paths = [image_dir + img_name for img_name in data[0]]
    scores = data[1].values
    return image_paths, scores

def create_dataloader(image_paths, scores, batch_size=64):
    images = []
    for img_path in image_paths:
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        print(image)
        images.append(image)

    images = torch.stack(images)
    scores = torch.tensor(scores, dtype=torch.float32)

    dataset = TensorDataset(images, scores)


    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [7]:
train_image_paths, train_scores = load_data(train_txt, image_dir)
test_image_paths, test_scores = load_data(test_txt, image_dir)

train_loader = create_dataloader(train_image_paths, train_scores)
test_loader = create_dataloader(test_image_paths, test_scores)

/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_48623/729663538.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(txt_file, delim_whitespace=True, header=None)
/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_48623/729663538.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(txt_file, delim_whitespace=True, header=None)


tensor([[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         ...,
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

        [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         ...,
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

        [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
         [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
         [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.

KeyboardInterrupt: 

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=True)
# model = torch.nn.DataParallel(model)

# # Set the model to run on the device
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)
model.load_state_dict(torch.load("resnet18_finetuned.pth"))
model.eval()
# model = model.to(device)



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/var/folders/_h/lfj08q4n53s2rb5h08cln0640000gn/T/ipykernel_43488/1933932580.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [37]:
def train_model(num_epochs = 10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        progress = 0
        for inputs, targets in train_loader:
            progress += 1
            print(f'Progress: {progress}/{len(train_loader)}', end='\r')
            optimizer.zero_grad()
            # inputs = inputs.to(device)
            # targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        

    model.eval()

def test_model():
    with torch.no_grad():
        running_loss = 0.0
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            running_loss += loss.item() * inputs.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

# train_model(10)
test_model()



Test Loss: 0.0859


In [ ]:
# torch.save(model.state_dict(), 'resnet18_finetuned.pth')


In [14]:
manual_check_dir = "ManualCheck/"
# print(os.listdir(manual_check_dir))

def PrintBeauty(dir_path):
    for img_path in os.listdir(dir_path):
        image = Image.open(dir_path + img_path).convert('RGB')
        image = transform(image)
        print(model(image.unsqueeze(0)).data[0][0])

PrintBeauty(manual_check_dir)



tensor(2.6751)


In [15]:
print(load("facial landmark/AF1.pts"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9a in position 10: invalid start byte